In [21]:
#########量化投资轮动策略#########
#based on Jupyter Notebook  
#version：python3.6
#liuhuanshuo：huanshuo0801@Gmail.com
#Apr,11,2018
####################

In [49]:
import pandas as pd
import time
import os
from pandas.core.frame import DataFrame

In [50]:
def riqi(begin,end):
    '将输入的开始和结束时间经过筛选给出需要计算的时间区间'
    
    a = pd.read_excel('/Users/liuhuanshuo/Desktop/pool_date_new.xlsx')
    
    k = 0
    
    for i in range(327): 
        a.ix[k,0] = a.ix[k,0].strftime("%Y-%m-%d")
        k = k+1
    
    s = list(a['日期']).index(begin)
    p = list(a['日期']).index(end)
    a.ix[s,2] = 'F' #修改第一天的end 不卖出
    a.ix[p,3] = 'F' #修改最后一天的select  不计算
    t = a[s:p+1].reset_index(drop=True) #计算区间
    t = t.fillna(0) 
    return t

In [51]:
def shouyilv(jisuanriqi,geshu):
    '用于计算时间内最后一个交易日的收益率前n个股票'
    
    temp = pd.DataFrame(columns = ['代码','日期','市盈率','开盘价(元)','收盘价(元)']) #储存代码
    for i in jisuanriqi['日期']:
        df_list = pd.read_excel('/Users/liuhuanshuo/Desktop/stocks_list.xlsx')
        temp1 = pd.DataFrame(columns = ['代码','日期','市盈率','开盘价(元)','收盘价(元)'])
        for file in df_list['股票代码']:
            excel_path = os.path.join('/Users','liuhuanshuo','desktop','作业','量化','单佳雷数据','沪A股票原始数据/') + file + '.xls'
            df = pd.read_excel(excel_path) 
            df_one = df.loc[:,['代码','日期','市盈率','开盘价(元)','收盘价(元)']]    #切片 提取指定列
            a = df_one[(df_one.日期 == i) & (df_one.市盈率 > 0)]  #选择特定行
            temp1 = temp1.append(a,ignore_index=True)    #将刚刚选取的行添加到数据框中
        temp1_sort = temp1.sort_values(by = '市盈率')  #排序
        s = temp1_sort.iloc[0:geshu]
        s = s.reset_index(drop=True)
        temp = temp.append(s,ignore_index=True)
    
    return temp

In [70]:
def mairuriqi(t,goumaidaima):
    mairuriqi = t[t['buy'] == 'T']
    temp3 = pd.DataFrame(columns = ['代码','日期','开盘价(元)','市盈率','收盘价(元)'])
    for i in mairuriqi['日期']:
        df_list = pd.read_excel('/Users/liuhuanshuo/Desktop/stocks_list.xlsx')
        for file in goumaidaima[0:geshu].代码:
            excel_path = os.path.join('/Users','liuhuanshuo','desktop','作业','量化','单佳雷数据','沪A股票原始数据/') + file + '.xls'
            df = pd.read_excel(excel_path) 
            df_one = df.loc[:,['代码','日期','开盘价(元)','市盈率','收盘价(元)']]    #切片 提取指定列
            a = df_one[(df_one.日期 == i) & (df_one.市盈率 > 0)]  #选择特定行
            temp3 = temp3.append(a,ignore_index=True)
    return temp3

In [71]:
def zhengchangriqi(t,goumaidaima):
    zhengchangriqi = t[t['buy'] == 0]
    temp4 = pd.DataFrame(columns = ['代码','日期','开盘价(元)','市盈率','收盘价(元)'])
    for i in zhengchangriqi['日期']:
        df_list = pd.read_excel('/Users/liuhuanshuo/Desktop/stocks_list.xlsx')
        for file in goumaidaima[0:geshu].代码:
            excel_path = os.path.join('/Users','liuhuanshuo','desktop','作业','量化','单佳雷数据','沪A股票原始数据/') + file + '.xls'
            df = pd.read_excel(excel_path) 
            df_one = df.loc[:,['代码','日期','开盘价(元)','市盈率','收盘价(元)']]    #切片 提取指定列
            a = df_one[(df_one.日期 == i) & (df_one.市盈率 > 0)]  #选择特定行
            temp4 = temp4.append(a,ignore_index=True)
     
    return temp4

In [75]:
def maichudaima(riqi_list,goumaidaima):
    maichudaima = riqi_list[riqi_list['end'] == 'T']
    temp2 = pd.DataFrame(columns = ['代码','日期','开盘价(元)','市盈率','收盘价(元)'])
    for i in maichudaima['日期']:
        df_list = pd.read_excel('/Users/liuhuanshuo/Desktop/stocks_list.xlsx')
        for file in goumaidaima[0:geshu].代码:
            excel_path = os.path.join('/Users','liuhuanshuo','desktop','作业','量化','单佳雷数据','沪A股票原始数据/') + file + '.xls'
            df = pd.read_excel(excel_path) 
            df_one = df.loc[:,['代码','日期','开盘价(元)','市盈率','收盘价(元)']]    #切片 提取指定列
            a = df_one[(df_one.日期 == i) & (df_one.市盈率 > 0)]  #选择特定行
            temp2 = temp2.append(a,ignore_index=True)
    return temp2

In [105]:
def lundong(t,temp2,temp3,temp4):
    '用于计算策略轮动'
    
    keyongzijin = cash
    chigujiazhi = 0
    keyongzijin_1 = []
    chigujiazhi_1 = []
    l = 0
    chigushuliang = []  #存储持股数量
    pp = 0
    kk = 0
    gg = 0
    for i in range(len(t)):

        if (t.ix[i,1] == 'T' and t.ix[i,2] == 'F'):
            goumaizijin = keyongzijin/geshu #第一次100万

            for l in range(geshu):
                chigushuliang_1 = (goumaizijin/temp3.ix[l+pp,2])
                chigushuliang.append(chigushuliang_1)

            keyongzijin = 0 #第一次卖完变成0
            chigujiazhi = goumaizijin * geshu
            pp = pp + geshu
            keyongzijin_1.append(keyongzijin)
            chigujiazhi_1.append(chigujiazhi)

        elif (t.ix[i,1] == 'F' and t.ix[i,2] == 'T'):
            chigujiazhi = 0
            chucunzijin = []
            for y in range(geshu):
                keyongzijin_2 = chigushuliang[y] * temp2.ix[y+kk,4]
                chucunzijin.append(keyongzijin_2)
            kk = kk + geshu
            keyongzijin = sum(chucunzijin)
            keyongzijin_1.append(keyongzijin)
            chigujiazhi_1.append(chigujiazhi)

        elif (t.ix[i,1] == 0 and t.ix[i,2] == 0 and t.ix[i,3] == 0):
            xianyoujiazhi = []
            for z in range(geshu):
                xianyoujiazhi_1 = chigushuliang[z] * temp4.ix[z + gg,2]
                xianyoujiazhi.append(xianyoujiazhi_1)
            gg = gg + geshu
            chigujiazhi = sum(xianyoujiazhi)
            keyongzijin_1.append(keyongzijin)
            chigujiazhi_1.append(chigujiazhi)
        else:
            keyongzijin_1.append(keyongzijin)
            chigujiazhi_1.append(chigujiazhi)

    c={"可用资金" : keyongzijin_1,
       "持股价值" : chigujiazhi_1}
    data1=DataFrame(c)
    data1.insert(0,'日期',t['日期'])
    
    return data1

In [106]:
def huitu():
    pass

In [107]:
def jisuan(begin,end):
    
    t = riqi(begin,end) #获得日期
    print(t)
    jisuanriqi = t[t['select'] == 'T'] #选择计算日期
    jisuanriqi = jisuanriqi.reset_index(drop=True)
    goumaidaima = shouyilv(jisuanriqi,geshu)
    print(goumaidaima)#获得需要购买的股票代码
    temp3 = mairuriqi(t,goumaidaima)
    temp4 = zhengchangriqi(t,goumaidaima)
    temp2 = maichudaima(t,goumaidaima)
    data = lundong(t,temp2,temp3,temp4)
    return data

In [114]:
begin =input('请输入策略开始时间：')
end = input('请输入策略结束时间：')
geshu = input('请输入每次选取的收益率排名：')
geshu = int(geshu)
cash = input('请输入策略本金：')
cash = int(cash)
result = jisuan(begin,end)
result

请输入策略开始时间：2016-02-05
请输入策略结束时间：2016-04-08
请输入每次选取的收益率排名：3
请输入策略本金：1000000


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if __name__ == '__main__':


            日期 buy end select
0   2016-02-05   F   F      T
1   2016-02-15   T   F      F
2   2016-02-16   0   0      0
3   2016-02-17   0   0      0
4   2016-02-18   0   0      0
5   2016-02-19   F   T      T
6   2016-02-22   T   F      F
7   2016-02-23   0   0      0
8   2016-02-24   0   0      0
9   2016-02-25   0   0      0
10  2016-02-26   F   T      T
11  2016-02-29   T   F      F
12  2016-03-01   0   0      0
13  2016-03-02   0   0      0
14  2016-03-03   0   0      0
15  2016-03-04   F   T      T
16  2016-03-07   T   F      F
17  2016-03-08   0   0      0
18  2016-03-09   0   0      0
19  2016-03-10   0   0      0
20  2016-03-11   F   T      T
21  2016-03-14   T   F      F
22  2016-03-15   0   0      0
23  2016-03-16   0   0      0
24  2016-03-17   0   0      0
25  2016-03-18   F   T      T
26  2016-03-21   T   F      F
27  2016-03-22   0   0      0
28  2016-03-23   0   0      0
29  2016-03-24   0   0      0
30  2016-03-25   F   T      T
31  2016-03-28   T   F      F
32  2016-0

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but 

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  from ipykernel import kernelapp as app
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:32: DeprecationWarning: 


,日期,可用资金,持股价值
0,2016-02-05,1.000000e+06,0.000000e+00
1,2016-02-15,0.000000e+00,1.000000e+06
2,2016-02-16,0.000000e+00,1.013155e+06
3,2016-02-17,0.000000e+00,1.024315e+06
4,2016-02-18,0.000000e+00,1.030147e+06
5,2016-02-19,1.021915e+06,0.000000e+00
6,2016-02-22,0.000000e+00,1.021915e+06
7,2016-02-23,0.000000e+00,1.032549e+06
8,2016-02-24,0.000000e+00,1.023439e+06
9,2016-02-25,0.000000e+00,1.026105e+06


In [115]:
begin =input('请输入策略开始时间：')
end = input('请输入策略结束时间：')
geshu = input('请输入每次选取的收益率排名：')
geshu = int(geshu)
cash = input('请输入策略本金：')
cash = int(cash)
result = jisuan(begin,end)
result

请输入策略开始时间：2016-04-01
请输入策略结束时间：2016-05-27
请输入每次选取的收益率排名：6
请输入策略本金：1000000


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if __name__ == '__main__':


            日期 buy end select
0   2016-04-01   F   F      T
1   2016-04-05   T   F      F
2   2016-04-06   0   0      0
3   2016-04-07   0   0      0
4   2016-04-08   F   T      T
5   2016-04-11   T   F      F
6   2016-04-12   0   0      0
7   2016-04-13   0   0      0
8   2016-04-14   0   0      0
9   2016-04-15   F   T      T
10  2016-04-18   T   F      F
11  2016-04-19   0   0      0
12  2016-04-20   0   0      0
13  2016-04-21   0   0      0
14  2016-04-22   F   T      T
15  2016-04-25   T   F      F
16  2016-04-26   0   0      0
17  2016-04-27   0   0      0
18  2016-04-28   0   0      0
19  2016-04-29   F   T      T
20  2016-05-03   T   F      F
21  2016-05-04   0   0      0
22  2016-05-05   0   0      0
23  2016-05-06   F   T      T
24  2016-05-09   T   F      F
25  2016-05-10   0   0      0
26  2016-05-11   0   0      0
27  2016-05-12   0   0      0
28  2016-05-13   F   T      T
29  2016-05-16   T   F      F
30  2016-05-17   0   0      0
31  2016-05-18   0   0      0
32  2016-0

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but 

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but 

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  from ipykernel import kernelapp as app
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:32: DeprecationWarning: 


,日期,可用资金,持股价值
0,2016-04-01,1.000000e+06,0.000000e+00
1,2016-04-05,0.000000e+00,1.000000e+06
2,2016-04-06,0.000000e+00,1.009162e+06
3,2016-04-07,0.000000e+00,1.007721e+06
4,2016-04-08,9.896844e+05,0.000000e+00
5,2016-04-11,0.000000e+00,9.896844e+05
6,2016-04-12,0.000000e+00,9.974851e+05
7,2016-04-13,0.000000e+00,9.985532e+05
8,2016-04-14,0.000000e+00,1.008977e+06
9,2016-04-15,1.009131e+06,0.000000e+00
